# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [5]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [6]:
def oracle010(qq):
    # YOUR CODE HERE
    yield X(qq[0])
    yield X(qq[2])
    yield Z(qq[2]).controlled_by(qq[0], qq[1])
    yield X(qq[2])
    yield X(qq[0])

In [7]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────@───────
          │
2: ───X───@───X───

In [8]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [9]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [10]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'010': 95, '100': 2, '000': 2, '011': 1})


95

In [11]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Write an oracle for the graph described below to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check for a valid coloring. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.



$G$ has 5 vertices. Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla.

The list of edges:
(0,3)
(0,4)
(1,3)
(1,4)
(2,3)
(2,4)


In [12]:
def oracle2(qq):
    # YOUR CODE HERE
    yield CX(qq[0], qq[5])
    yield CX(qq[3], qq[5])
    yield CX(qq[0], qq[6])
    yield CX(qq[4], qq[6])
    yield CX(qq[1], qq[7])
    yield CX(qq[3], qq[7])
    yield CX(qq[1], qq[8])
    yield CX(qq[4], qq[8])
    yield CX(qq[2], qq[9])
    yield CX(qq[3], qq[9])
    yield CX(qq[2], qq[10])
    yield CX(qq[4], qq[10])

    yield X(qq[11]).controlled_by(*qq[5:11])

In [13]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [14]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [15]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'11100': 450,
         '00011': 444,
         '01011': 7,
         '01101': 6,
         '11010': 6,
         '00001': 6,
         '00100': 6,
         '11000': 6,
         '11111': 5,
         '10010': 5,
         '01111': 4,
         '01100': 4,
         '01110': 4,
         '11110': 4,
         '00110': 4,
         '00111': 4,
         '00010': 4,
         '01001': 3,
         '10110': 3,
         '10001': 3,
         '01010': 2,
         '10100': 2,
         '10101': 2,
         '00000': 2,
         '00101': 2,
         '11001': 2,
         '10111': 2,
         '10011': 2,
         '01000': 2,
         '10000': 2,
         '11101': 1,
         '11011': 1})

In [16]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Write an oracle for the graph described below to check whether there exists a coloring with 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 14-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check whether there exists a coloring with 4 edges connecting vertices with different colors. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.


$G$ has 4 vertices. Use qubits 0-4 for the vertices, 5-9 for the edges, 10-12 for the addition and 13 as the ancilla.

The list of edges: 
(0,1)
(0,2)
(0,3)
(1,2)
(1,3)

In [17]:
def oracle3(qq):
    # YOUR CODE HERE
    yield CX(qq[0], qq[5])
    yield CX(qq[1], qq[5])
    yield CX(qq[0], qq[6])
    yield CX(qq[2], qq[6])
    yield CX(qq[0], qq[7])
    yield CX(qq[3], qq[7])
    yield CX(qq[1], qq[8])
    yield CX(qq[2], qq[8])
    yield CX(qq[1], qq[9])
    yield CX(qq[3], qq[9])

    yield X(qq[10]).controlled_by(qq[5], qq[6], qq[7], qq[8])
    yield X(qq[10]).controlled_by(qq[5], qq[6], qq[7], qq[9])
    yield X(qq[10]).controlled_by(qq[5], qq[6], qq[8], qq[9])
    yield X(qq[10]).controlled_by(qq[5], qq[7], qq[8], qq[9])
    yield X(qq[10]).controlled_by(qq[6], qq[7], qq[8], qq[9])


    yield CX(qq[10], qq[13])

In [18]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[13])
    yield inverse(oracle3(qq))  

In [19]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(14)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [20]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'1100': 473,
         '0011': 455,
         '0110': 14,
         '0111': 9,
         '0001': 8,
         '1010': 7,
         '1011': 6,
         '1001': 4,
         '1110': 4,
         '1101': 4,
         '0101': 3,
         '0010': 3,
         '0000': 3,
         '1111': 3,
         '1000': 2,
         '0100': 2})

In [21]:
# hidden tests in this cell will be used for grading.